In [10]:
import tensorflow as tf

In [12]:
tf.one_hot([0, 0], depth=2)

<tf.Tensor: id=8, shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.]], dtype=float32)>

## import gym

In [2]:
env_name = "CartPole-v0"
env = gym.make(env_name)

In [9]:
env.action_space.n

2

env_name = "CartPole-v0" <br>
x_ph.size = 4 <br>
x2_ph.size = 4 <br>
a_ph.size = 1 <br>

actor_critic(x_ph, a_ph, alpha, action_space=env.action_space)